In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：

### 目标函数：
bounds = [(0,1000),(0,40),(0,40),(100,300),(6.3,6.7),(5.9,6.4),(4.5,6.25)]
x_opt = [193.724510070034967, 5.56944131553368433e-27, 17.3191887294084914, 100.047897801386839, 
         6.68445185362377892, 5.99168428444264833, 6.21451648886070451]
y_opt = 193.724510070035
def G21(X):
    
    y = X[0]
    g1 = -X[0]+35*X[1]**0.6+35*X[2]**0.6
    h1 = abs(-300*X[2]+7500*X[4]-7500*X[5]-25*X[3]*X[4]+25*X[3]*X[5]+X[2]*X[3])-0.0001
    #h11 = -(-300*X[2]+7500*X[4]-7500*X[5]-25*X[3]*X[4]+25*X[3]*X[5]+X[2]*X[3])-0.0001
    h2 = abs(100*X[1]+155.365*X[3]+2500*X[6]-X[1]*X[3]-25*X[3]*X[6]-15536.5)-0.0001
    #h22 = -(100*X[1]+155.365*X[3]+2500*X[6]-X[1]*X[3]-25*X[3]*X[6]-15536.5)-0.0001
    h3 = abs(-X[4]+np.log(-X[3]+900))-0.0001
    #h33 = -(-X[4]+np.log(-X[3]+900))-0.0001
    h4 = abs(-X[5]+np.log(X[3]+300))-0.0001
    #h44 = -(-X[5]+np.log(X[3]+300))-0.0001
    h5 = abs(-X[6]+np.log(-2*X[3]+700))-0.0001
    #h55 = -(-X[6]+np.log(-2*X[3]+700))-0.0001
    
    return y,g1,h1,h2,h3,h4,h5
#     return y,g1,h1,h11,h2,h22,h3,h33,h4,h44,h5,h55

In [4]:
# G21(x_opt)

In [5]:
np.log(800)

6.684611727667927

In [6]:
g1 = lambda X: G21(X)[1]
h1 = lambda X: G21(X)[2]
h2 = lambda X: G21(X)[3]
h3 = lambda X: G21(X)[4]
h4 = lambda X: G21(X)[5]
h5 = lambda X: G21(X)[6]
x0 = [0,0,40,300,6.3,5.9,6.24]

In [7]:
maxfunction(h2,x0,bounds)

     fun: -8176.9999
     jac: array([    0.        ,  -200.        ,    -0.        ,   -40.88476562,
           0.        ,     0.        , -5000.        ])
 message: 'Optimization terminated successfully'
    nfev: 16
     nit: 3
    njev: 2
  status: 0
 success: True
       x: array([  0.  ,  40.  ,  40.  , 300.  ,   6.3 ,   5.9 ,   6.25])

In [8]:
640.2270726982026+8500+8572.9999+3

17716.226972698205

In [9]:
### 适应度函数
def fitnessG21_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1220.174
    fmin = 0.008509619375000003
    count = np.sum(np.array(problem1(X))[1:5]>0)
    
    return -(problem1(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG21_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1000
    fmin = 0
    vmax = 17716.226972698205
    e = 0.000000001 
    n = 6 
    
    res = G21(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:7]>0)
    
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness    

In [10]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-10.0e+28)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-10.0e+28)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG21_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/800))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
            w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG21_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/800))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit
    

In [11]:
PSO = PSO_fit(pN=8000, dim=7, max_iter=1000, bounds=bounds)

In [12]:
PSO.init_Population()

In [13]:
PSO.iterator()

[-2.50879451e+10 -1.78125133e+10 -3.08159373e+10 ... -1.01009484e+10
 -2.46077225e+10 -3.91304267e+10]
[-1.01009484e+10 -1.01009484e+10 -3.08159373e+10 ... -1.01009483e+10
 -1.01009484e+10 -1.01009479e+10]
[-1.01009482e+10 -1.01009475e+10 -1.01009483e+10 ... -9.72756455e+09
 -1.01009480e+10 -1.01009479e+10]
[-3.14978435e+08 -1.01009475e+10 -1.01009482e+10 ... -9.26028825e+09
 -1.01009480e+10 -1.01009474e+10]
[-3.14978435e+08 -1.01009474e+10 -1.01009474e+10 ... -9.26028825e+09
 -3.95665209e+09 -7.03670702e+09]
[-3.14978435e+08 -8.97844559e+09 -1.23247980e+09 ... -9.26028825e+09
 -2.73154676e+09 -7.03670702e+09]
[-3.14978435e+08 -8.20490644e+09 -1.23247980e+09 ... -8.46001389e+09
 -2.73154676e+09 -7.03670702e+09]
[-3.14978435e+08 -5.93991180e+08 -1.23247980e+09 ... -1.73971132e+09
 -7.07533025e+08 -1.41901905e+09]
[-3.14978435e+08 -5.93991180e+08 -1.23247980e+09 ... -1.50515926e+09
 -7.07533025e+08 -1.41901905e+09]
[-2.75985546e+08 -5.93991180e+08 -1.23247980e+09 ... -1.50515926e+09
 -7.

[-27590498.94816225 -25955758.91946518 -26903324.3307979  ...
 -26894140.12641483 -27596656.84852972 -27468946.94742968]
[-26023111.38176566 -25955758.91946518 -26903324.3307979  ...
 -26894140.12641483 -27596656.84852972 -27468946.94742968]
[-26023111.38176566 -25955758.91946518 -26903324.3307979  ...
 -26894140.12641483 -27596656.84852972 -27151725.09970081]
[-26023111.38176566 -25955758.91946518 -26903324.3307979  ...
 -26894140.12641483 -27596656.84852972 -27151725.09970081]
[-26023111.38176566 -25955758.91946518 -26903324.3307979  ...
 -26894140.12641483 -27596656.84852972 -27151725.09970081]
[-26023111.38176566 -25955758.91946518 -26903324.3307979  ...
 -26894140.12641483 -27596656.84852972 -27151725.09970081]
[-26023111.38176566 -25955758.91946518 -26903324.3307979  ...
 -26894140.12641483 -26103046.55584671 -27151725.09970081]
[-26023111.38176566 -25955758.91946518 -26903324.3307979  ...
 -26894140.12641483 -26041654.59580765 -27151725.09970081]
[-26023111.38176566 -25955758.91

[-24249344.82565107 -24039668.44314251 -24355040.54644492 ...
 -24149112.93885792 -24489515.76613022 -23987481.07861156]
[-24249344.82565107 -24039668.44314251 -24355040.54644492 ...
 -24149112.93885792 -24489515.76613022 -23987481.07861156]
[-24249344.82565107 -24039668.44314251 -24355040.54644492 ...
 -24149112.93885792 -24489515.76613022 -23987481.07861156]
[-24249344.82565107 -24039668.44314251 -24267521.54838689 ...
 -24149112.93885792 -24489515.76613022 -23942068.09836723]
[-24249344.82565107 -24039668.44314251 -24026888.80328925 ...
 -24149112.93885792 -24489515.76613022 -23942068.09836723]
[-24249344.82565107 -24039668.44314251 -24026888.80328925 ...
 -24149112.93885792 -24489515.76613022 -23942068.09836723]
[-24249344.82565107 -24039668.44314251 -24026888.80328925 ...
 -24149112.93885792 -24489515.76613022 -23942068.09836723]
[-23960712.68941515 -24039668.44314251 -24026888.80328925 ...
 -24108659.16820968 -24489515.76613022 -23942068.09836723]
[-23960712.68941515 -23993018.07

[-20834466.09270958 -20778071.92078878 -20630569.57617989 ...
 -20650279.45438853 -20856681.1687489  -20630569.57617989]
[-20834466.09270958 -20778071.92078878 -20630569.57617989 ...
 -20650279.45438853 -20856681.1687489  -20630569.57617989]
[-20834466.09270958 -20778071.92078878 -20630569.57617989 ...
 -20650279.45438853 -20856681.1687489  -20630569.57617989]
[-20834466.09270958 -20778071.92078878 -20630569.57617989 ...
 -20650279.45438853 -20856681.1687489  -20630569.57617989]
[-20834466.09270958 -20778071.92078878 -20630569.57617989 ...
 -20650279.45438853 -20721697.43116516 -20630569.57617989]
[-20834466.09270958 -20778071.92078878 -20630569.57617989 ...
 -20650279.45438853 -20721697.43116516 -20630569.57617989]
[-20834466.09270958 -20778071.92078878 -20630569.57617989 ...
 -20650279.45438853 -20721697.43116516 -20630569.57617989]
[-20834466.09270958 -20778071.92078878 -20630569.57617989 ...
 -20650279.45438853 -20721697.43116516 -20630569.57617989]
[-20834466.09270958 -20778071.92

[-15654481.24530412  -9031775.7640533  -17938815.53297299 ...
 -13383617.28655813 -12981951.14498108  -9156361.80876997]
[-15654481.24530412  -9031775.7640533  -17938815.53297299 ...
 -13383617.28655813  -8823148.9336241   -9156361.80876997]
[ -7328776.72501288  -9031775.7640533   -6646957.09228977 ...
 -13383617.28655813  -8823148.9336241   -9156361.80876997]
[ -7328776.72501288  -9031775.7640533   -6646957.09228977 ...
 -13383617.28655813  -8823148.9336241   -9156361.80876997]
[ -7328776.72501288  -9031775.7640533   -6646957.09228977 ...
 -13383617.28655813  -8823148.9336241   -9156361.80876997]
[ -7328776.72501288  -7265956.45206083  -6356451.56894685 ...
 -13383617.28655813  -8823148.9336241   -9156361.80876997]
[ -7328776.72501288  -7265956.45206083  -6356451.56894685 ...
 -13383617.28655813  -8823148.9336241   -9156361.80876997]
[ -7328776.72501288  -7265956.45206083  -6356451.56894685 ...
 -13383617.28655813  -8823148.9336241   -9156361.80876997]
[ -7328776.72501288  -7265956.45

...

[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]

[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]

[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]

[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]
[-193.75464427 -193.75464427 -193.75464427 ... -193.75464427 -193.75464427
 -193.75464427]

(array([[1.93754644e+02, 0.00000000e+00, 1.73236790e+01, ...,
         6.68451173e+00, 5.99156455e+00, 6.21456695e+00],
        [1.93754644e+02, 0.00000000e+00, 1.73236790e+01, ...,
         6.68451173e+00, 5.99156455e+00, 6.21457464e+00],
        [1.93754644e+02, 0.00000000e+00, 1.73236790e+01, ...,
         6.68451173e+00, 5.99156455e+00, 6.21460801e+00],
        ...,
        [1.93754644e+02, 0.00000000e+00, 1.73236790e+01, ...,
         6.68451173e+00, 5.99156455e+00, 6.21456391e+00],
        [1.93754644e+02, 2.15749914e-27, 1.73236790e+01, ...,
         6.68451173e+00, 5.99156455e+00, 6.21451485e+00],
        [1.93754644e+02, 0.00000000e+00, 1.73236790e+01, ...,
         6.68451173e+00, 5.99156455e+00, 6.21456148e+00]]),
 array([-193.75464427, -193.75464427, -193.75464427, ..., -193.75464427,
        -193.75464427, -193.75464427]))

In [14]:
G21(PSO.gbest[0])

(193.7546442672643,
 0.0,
 -2.5102058561106422e-14,
 -0.0001,
 -1.1212412292030405e-15,
 -2.3306280950291525e-16,
 -5.8848499216124144e-05)

In [15]:
PSO.gbest[0]

array([193.75464427,   0.        ,  17.32367901, 100.        ,
         6.68451173,   5.99156455,   6.21456695])

In [16]:
PSO.gbest[1]

array([193.75464427,   0.        ,  17.32367901, 100.        ,
         6.68451173,   5.99156455,   6.21457464])

In [24]:
np.random.random()

0.7471256803570261